In [4]:
from ultralytics import YOLO

## Print model info

In [ ]:
model = YOLO('yolov8n.pt')
model.info()

100%|██████████| 6.23M/6.23M [00:01<00:00, 4.43MB/s]


YOLOv8n summary: 225 layers, 3157200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)

## Simple Load, Train and Infer
Typical script

In [5]:
%%script false --no-raise-error

# Create a new YOLO model from scratch
model = YOLO('yolov8n.yaml')

# Load a COCO-pretrained YOLOv8n model
model = YOLO('yolov8n.pt')


# Display model information (optional)
model.info()

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data='coco8.yaml', epochs=100, imgsz=640)

# Evaluate the model's performance on the validation set
results = model.val()

# Run inference with the YOLOv8n model on the 'bus.jpg' image
results = model('path/to/bus.jpg')

# Export the model to ONNX format
success = model.export(format='onnx', dynamic=True)

## Inference
### Predict

In [7]:
%%script false --no-raise-error

from ultralytics import YOLO
from PIL import Image
import cv2

model = YOLO("model.pt")
# accepts all formats - image/dir/Path/URL/video/PIL/ndarray. 0 for webcam
results = model.predict(source="0")
results = model.predict(source="folder", show=True) # Display preds. Accepts all YOLO predict arguments

# from PIL
im1 = Image.open("bus.jpg")
results = model.predict(source=im1, save=True)  # save plotted images

# from ndarray
im2 = cv2.imread("bus.jpg")
results = model.predict(source=im2, save=True, save_txt=True)  # save predictions as labels

# from list of PIL/ndarray
results = model.predict(source=[im1, im2])

In [ ]:
%%script false --no-raise-error

from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # pretrained YOLOv8n model

# Run batched inference on a list of images
results = model(['im1.jpg', 'im2.jpg'])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename='result.jpg')  # save to disk

## Exploring YOLO Repository

### yaml
Typical yaml file

### ultralytics/ultralytics/models/yolo/classify/train.py

In [1]:
%%script false --no-raise-error

from ultralytics.engine.trainer import BaseTrainer
...

class ClassificationTrainer(BaseTrainer):
    """
    A class extending the BaseTrainer class for training based on a classification model.

    Notes:
        - Torchvision classification models can also be passed to the 'model' argument, i.e. model='resnet18'.

    Example:
        ```python
        from ultralytics.models.yolo.classify import ClassificationTrainer

        args = dict(model='yolov8n-cls.pt', data='imagenet10', epochs=3)
        trainer = ClassificationTrainer(overrides=args)
        trainer.train()
        ```
    """
    ...

### Callbacks
#### Returning additional information with Prediction

In this example, we want to return the original frame with each result object. Here's how we can do that



In [ ]:
%%script false --no-raise-error

from ultralytics import YOLO


def on_predict_batch_end(predictor):
    """Handle prediction batch end by combining results with corresponding frames; modifies predictor results."""
    _, image, _, _ = predictor.batch

    # Ensure that image is a list
    image = image if isinstance(image, list) else [image]

    # Combine the prediction results with the corresponding frames
    predictor.results = zip(predictor.results, image)


# Create a YOLO model instance
model = YOLO(f'yolov8n.pt')

# Add the custom callback to the model
model.add_callback("on_predict_batch_end", on_predict_batch_end)

# Iterate through the results and frames
for (result, frame) in model.predict():  # or model.track()
    pass